In [1]:
import pandas as pd
import numpy as np
from nilearn import plotting
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


load link matrix

In [2]:
svm_link = pd.read_csv('./link_matrix/link_count_SVM.csv')
# Selection frequency>1000
svm_link = svm_link[svm_link['link']>1000]

Fetch node

In [3]:
node_node = svm_link['Unnamed: 0']
# All the nodes connecting the brain regions are extracted
node = []
for i in range(len(node_node)):
    a, b = node_node[i].split('_')[0], node_node[i].split('_')[1]
    node.append(a)
    node.append(b)
# The node name is deduplicated
node_set = list(set(node))

matrix

In [18]:
# The link table is converted to a node*node matrix
def link_node_matrix(svm_link):
    # Name of brain region connection
    node_node = list(svm_link['Unnamed: 0'])
    # Standard
    svm_link['standard'] = ""
    linkValue = np.array(svm_link['link'])
    linkValue_std = (linkValue - linkValue.min())/(linkValue.max() - linkValue.min())
    svm_link['standard'] = list(linkValue_std)
    # All the nodes connecting the brain regions are extracted
    node = []
    for i in range(len(node_node)):
        a, b = node_node[i].split('_')[0], node_node[i].split('_')[1]
        node.append(a)
        node.append(b)
    # The node name is deduplicated
    node_set = list(set(node))
    # Create a new empty table for node*node
    link_matrix = pd.DataFrame(columns=node_set, index=node_set)
    # Fill in the empty table with the corresponding values for brain connections
    for i in range(len(node_node)):
        a, b = node_node[i].split('_')[0], node_node[i].split('_')[1]
        link_matrix.loc[a, b] = svm_link['standard'][i]
        link_matrix.loc[b, a] = svm_link['standard'][i]
        link_matrix = link_matrix.fillna(0)
        
    return node_set, link_matrix

In [23]:
node_set, link_matrix = link_node_matrix(svm_link)

Coordinate table making

In [7]:
# Coordinate
axis = pd.read_excel('./coordinate/aal90_atlas.xlsx')

In [32]:
def node_coor(link_matrix, axis):
    node_columns = pd.DataFrame(link_matrix.columns, columns=['num'])
    node_columns['num'] = node_columns['num'].astype(int)
    node_coor = pd.merge(node_columns, axis, how='inner', on=['num'])
    node_coor = np.array(node_coor[['x', 'y', 'z']])
    
    return node_coor

In [24]:
node_coor = node_coor(link_matrix, axis)

The corresponding network of brain regions

In [10]:
network = pd.read_csv('./network/network.csv')

In [33]:
def node_color(link_matrix, network):
    node_columns = pd.DataFrame(link_matrix.columns, columns=['num'])
    node_columns['num'] = node_columns['num'].astype(int)
    node_color = pd.merge(node_columns, network, how='inner', on=['num'])
    node_color = node_color[['network', 'node_color']]
    
    return node_color

In [25]:
node_color = node_color(link_matrix, network)

Node size

In [34]:
# Add the values of each column to get the total join values of the node
def node_size(link_matrix):
    node_weight = np.array(link_matrix.sum(axis=0))
    node_size = 20 + 50 * node_weight
    
    return node_size

In [26]:
node_size = node_size(link_matrix)

Summarize all the information in one table

In [15]:
nilearn_svm_info = pd.DataFrame(columns=['node', 'x', 'y', 'z', 'network', 'node_color', 'node_size'])
nilearn_svm_info['node'] = node_set
nilearn_svm_info['node'] = nilearn_svm_info['node'].astype(int)
nilearn_svm_info['x'] = node_coor[:, 0]
nilearn_svm_info['y'] = node_coor[:, 1]
nilearn_svm_info['z'] = node_coor[:, 2]
nilearn_svm_info['network'] = list(node_color['network'])
nilearn_svm_info['node_color'] = list(node_color['node_color'])
nilearn_svm_info['node_size'] = node_size

In [16]:
name = axis[['num', 'abbr', 'name']]
name = name.rename(columns={'num': 'node'})
name['node'] = name['node'].astype(int)
nilearn_svm_info = pd.merge(nilearn_svm_info, name, how='inner', on=['node'])
nilearn_svm_info = nilearn_svm_info[['node', 'abbr', 'name', 'network', 'node_color', 'node_size', 'x', 'y', 'z']]

plotting.plot_connectome

In [27]:
fig = plt.figure()
plotting.plot_connectome(link_matrix, node_coor, edge_threshold=0.1, node_color=list(node_color['node_color']), node_size=node_size, colorbar=True, 
                         figure=fig, title='SVM', output_file='./svm_plot.pdf')  